In [ ]:
import tweepy
import csv

consumer_key = "**************************"
consumer_secret = "**************************"
access_key = "**************************"
access_secret = "**************************"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

# ツイート取得
tweet_data = []

for tweet in tweepy.Cursor(api.user_timeline,screen_name = "@takapon_jp",exclude_replies = True).items():
    tweet_data.append([tweet.id,tweet.created_at,tweet.text.replace('\n','')])

# csv出力
with open('./6050_stock_price_prediction_data/tweets_horie.csv', 'w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["id","text","created_at"])
    writer.writerows(tweet_data)

import re
import pandas as pd
import numpy  as np
import MeCab

df_tweets = pd.read_csv('./6050_stock_price_prediction_data/tweets_horie.csv', names=['id', 'date', 'text'], index_col='date')
df_tweets = df_tweets[['text']].sort_index(ascending=True)

# 辞書のデータフレームを作成．パスは適宜設定してください．
pn_df = pd.read_csv('./6050_stock_price_prediction_data/pn_ja.csv', encoding='utf-8', names=('Word','Reading','POS', 'PN'))
# 各列の分割
word_list = list(pn_df['Word'])
pn_list   = list(pn_df['PN'])
pn_dict   = dict(zip(word_list, pn_list))

# MeCabインスタンスの作成．引数を無指定にするとIPA辞書になります．
m = MeCab.Tagger('')

#NANを排除
df_tweets = df_tweets.dropna()

# テキストを形態素解析し辞書のリストを返す関数
def get_diclist(text):
    parsed = m.parse(text)      # 形態素解析結果（改行を含む文字列として得られる）
    lines = parsed.split('\n')  # 解析結果を1行（1語）ごとに分けてリストにする
    lines = lines[0:-2]         # 後ろ2行は不要なので削除
    diclist = []
    for word in lines:
        l = re.split('\t|,',word)  # 各行はタブとカンマで区切られてるので
        d = {'Surface':l[0], 'POS1':l[1], 'POS2':l[2], 'BaseForm':l[7]}
        diclist.append(d)
    return(diclist)

# 形態素解析結果の単語ごとのdictデータにPN値を追加する関数
def add_pnvalue(diclist_old, pn_dict):
    diclist_new = []
    for word in diclist_old:
        base = word['BaseForm']        # 個々の辞書から基本形を取得
        if base in pn_dict:
            pn = float(pn_dict[base]) 
        else:
            pn = 'notfound'            # その語がPN Tableになかった場合
        word['PN'] = pn
        diclist_new.append(word)
    return(diclist_new)

# 各ツイートのPN平均値を求める
def get_mean(dictlist):
    pn_list = []
    for word in dictlist:
        pn = word['PN']
        if pn!='notfound':
            pn_list.append(pn)
    if len(pn_list)>0:
        pnmean = np.mean(pn_list)
    else:
        pnmean=0
    return pnmean

means_list = []
for tweet in df_tweets['text']:
    dl_old = get_diclist(tweet)
    dl_new = add_pnvalue(dl_old, pn_dict)
    pnmean = get_mean(dl_new)
    means_list.append(pnmean)
means_list = np.copy(means_list)
x_std = (means_list - means_list.mean()) / means_list.std()
df_tweets['pn'] = x_std
df_tweets = df_tweets.drop('text', axis=0)
df_tweets.index = pd.to_datetime(df_tweets.index)
df_tweets =  df_tweets.resample('D', how='mean')
x = df_tweets.index
y = df_tweets.pn
plt.plot(x,y)
plt.grid(True)